In [1]:
import pandas as pd
import numpy as np

In [2]:
fin1 = pd.read_csv("Finland1_raw.csv")
fin2 = pd.read_csv("Finland2_raw.csv")

C:\Users\Big City Girl\AppData\Local\Temp\ipykernel_20024\2175392655.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  fin1 = pd.read_csv("Finland1_raw.csv")
C:\Users\Big City Girl\AppData\Local\Temp\ipykernel_20024\2175392655.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  fin2 = pd.read_csv("Finland2_raw.csv")


In [3]:
fin1.head()

,Information,"Underlying cause of death (ICD-10, 3-character level)",Age,Year,Gender,Deaths
0,Deaths,A00-Y89 Total,0,2008,Total,159
1,Deaths,A00-Y89 Total,0,2008,Males,94
2,Deaths,A00-Y89 Total,0,2008,Females,65
3,Deaths,A00-Y89 Total,0,2009,Total,160
4,Deaths,A00-Y89 Total,0,2009,Males,78


## Data cleaning

### 1. Adding fin1 and fin2 datasets together

In [4]:
fin = pd.concat([fin1, fin2])
fin.tail()

,Information,"Underlying cause of death (ICD-10, 3-character level)",Age,Year,Gender,Deaths
771493,Deaths,Y89 Sequelae of other external causes,95 -,2019,Males,0
771494,Deaths,Y89 Sequelae of other external causes,95 -,2019,Females,0
771495,Deaths,Y89 Sequelae of other external causes,95 -,2020,Total,0
771496,Deaths,Y89 Sequelae of other external causes,95 -,2020,Males,0
771497,Deaths,Y89 Sequelae of other external causes,95 -,2020,Females,0


### 2. Clean fin dataset

In [5]:
# drop unnecessary data
fin = fin.drop("Information",axis=1)

# rename column name
fin["Country"] ="Finland"
fin.columns = ["Cause", "Age range", "Year", "Gender", "Deaths", "Country"]

In [6]:
fin = fin[["Country", "Year", "Gender", "Age range", "Cause", "Deaths"]]
fin.head()

,Country,Year,Gender,Age range,Cause,Deaths
0,Finland,2008,Total,0,A00-Y89 Total,159
1,Finland,2008,Males,0,A00-Y89 Total,94
2,Finland,2008,Females,0,A00-Y89 Total,65
3,Finland,2009,Total,0,A00-Y89 Total,160
4,Finland,2009,Males,0,A00-Y89 Total,78


In [7]:
fin.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1415232 entries, 0 to 771497
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   Country    1415232 non-null  object
 1   Year       1415232 non-null  int64 
 2   Gender     1415232 non-null  object
 3   Age range  1415232 non-null  object
 4   Cause      1415232 non-null  object
 5   Deaths     1415232 non-null  object
dtypes: int64(1), object(5)
memory usage: 75.6+ MB


In [8]:
# change data type of "Deaths" variable
fin["Deaths"]=pd.to_numeric(fin.Deaths, errors='coerce').convert_dtypes() 
fin.head()

,Country,Year,Gender,Age range,Cause,Deaths
0,Finland,2008,Total,0,A00-Y89 Total,159
1,Finland,2008,Males,0,A00-Y89 Total,94
2,Finland,2008,Females,0,A00-Y89 Total,65
3,Finland,2009,Total,0,A00-Y89 Total,160
4,Finland,2009,Males,0,A00-Y89 Total,78


In [9]:
fin.isnull().sum()

Country          0
Year             0
Gender           0
Age range        0
Cause            0
Deaths       70434
dtype: int64

According to ICD code, we should narrow the death causes into 22 categories, which has been described in the excel file in _process folder. The total death number should be kept still. Let's remove all unneccesary values from "Cause" variable.

In [10]:
kept_list =['A00-Y89 Total']
fin = fin[fin.Cause.isin(kept_list) == True]
fin.reset_index(inplace=True)

In [11]:
fin.Cause.unique()

array(['A00-Y89 Total'], dtype=object)

In [12]:
fin.isnull().sum()
print(f"Deaths column has now only {fin.Deaths.isnull().sum()} null values, then we do not need to worry too much on these null values for now.")

Deaths column has now only 0 null values, then we do not need to worry too much on these null values for now.


In [13]:
fin["Age range"].unique()

array(['0', '1 - 4', '5 - 9', '10 - 14', '15 - 19', '20 - 24', '25 - 29',
       '30 - 34', '35 - 39', '40 - 44', '45 - 49', '50 - 54', '55 - 59',
       '60 - 64', '65 - 69', '70 - 74', '75 - 79', '80 - 84', '85 - 89',
       '90 - 94', '95 -'], dtype=object)

In [14]:
# modify age range so that matches ones in Germany dataset
fin_age_dict ={
    '0'      : '0 - 14',
    '1 - 4'  : '0 - 14',
    '5 - 9'  : '0 - 14',
    '10 - 14': '0 - 14',
    '15 - 19': '15 - 19',
    '20 - 24': '20 - 24',
    '25 - 29': '25 - 29',
    '30 - 34': '30 - 34', 
    '35 - 39': '35 - 39', 
    '40 - 44': '40 - 44', 
    '45 - 49': '45 - 49', 
    '50 - 54': '50 - 54', 
    '55 - 59': '55 - 59',
    '60 - 64': '60 - 64', 
    '65 - 69': '65 - 69', 
    '70 - 74': '70 - 74', 
    '75 - 79': '75 - 79', 
    '80 - 84': '80 - 84',
    '85 - 89': '85 - over',
    '90 - 94': '85 - over',
    '95 -'   : '85 - over'
}
fin['AgeRange_modified'] = fin['Age range'].map(fin_age_dict)
fin.head(10)

,index,Country,Year,Gender,Age range,Cause,Deaths,AgeRange_modified
0,0,Finland,2008,Total,0,A00-Y89 Total,159,0 - 14
1,1,Finland,2008,Males,0,A00-Y89 Total,94,0 - 14
2,2,Finland,2008,Females,0,A00-Y89 Total,65,0 - 14
3,3,Finland,2009,Total,0,A00-Y89 Total,160,0 - 14
4,4,Finland,2009,Males,0,A00-Y89 Total,78,0 - 14
5,5,Finland,2009,Females,0,A00-Y89 Total,82,0 - 14
6,6,Finland,2010,Total,0,A00-Y89 Total,138,0 - 14
7,7,Finland,2010,Males,0,A00-Y89 Total,81,0 - 14
8,8,Finland,2010,Females,0,A00-Y89 Total,57,0 - 14
9,9,Finland,2011,Total,0,A00-Y89 Total,142,0 - 14


In [15]:
fin["AgeRange_modified"].unique()
fin.head(
)

,index,Country,Year,Gender,Age range,Cause,Deaths,AgeRange_modified
0,0,Finland,2008,Total,0,A00-Y89 Total,159,0 - 14
1,1,Finland,2008,Males,0,A00-Y89 Total,94,0 - 14
2,2,Finland,2008,Females,0,A00-Y89 Total,65,0 - 14
3,3,Finland,2009,Total,0,A00-Y89 Total,160,0 - 14
4,4,Finland,2009,Males,0,A00-Y89 Total,78,0 - 14


In [16]:
fin = fin.groupby(["Year", "Gender", "AgeRange_modified"]).sum("Deaths").reset_index()
fin.head()

,Year,Gender,AgeRange_modified,index,Deaths
0,2008,Females,0 - 14,242,106
1,2008,Females,15 - 19,158,38
2,2008,Females,20 - 24,197,53
3,2008,Females,25 - 29,236,62
4,2008,Females,30 - 34,275,64


In [17]:
fin_new = fin.sort_values(["AgeRange_modified"],ascending=False).groupby(["Year", "Gender","AgeRange_modified"]).Deaths.sum()
fin_new

Year  Gender   AgeRange_modified
2008  Females  0 - 14                 106
               15 - 19                 38
               20 - 24                 53
               25 - 29                 62
               30 - 34                 64
                                    ...  
2020  Total    65 - 69               4089
               70 - 74               6482
               75 - 79               6401
               80 - 84               8844
               85 - over            21933
Name: Deaths, Length: 624, dtype: Int64

In [18]:
fin = fin.drop(['index'], axis=1)

In [19]:
fin["Country"] = "Finland"
fin.columns = ["Year", "Gender", "Age Range", "Deaths", "Country"]
fin.head()

,Year,Gender,Age Range,Deaths,Country
0,2008,Females,0 - 14,106,Finland
1,2008,Females,15 - 19,38,Finland
2,2008,Females,20 - 24,53,Finland
3,2008,Females,25 - 29,62,Finland
4,2008,Females,30 - 34,64,Finland


In [20]:
fin = fin.loc[fin.Gender != "Total"].reset_index()
fin.head()

,index,Year,Gender,Age Range,Deaths,Country
0,0,2008,Females,0 - 14,106,Finland
1,1,2008,Females,15 - 19,38,Finland
2,2,2008,Females,20 - 24,53,Finland
3,3,2008,Females,25 - 29,62,Finland
4,4,2008,Females,30 - 34,64,Finland


In [21]:
fin = fin.drop(["index"], axis=1)

In [22]:
fin['Total'] = fin.groupby(["Year", "Age Range"], sort=False)['Deaths'].transform('sum')
fin.head()

,Year,Gender,Age Range,Deaths,Country,Total
0,2008,Females,0 - 14,106,Finland,265
1,2008,Females,15 - 19,38,Finland,138
2,2008,Females,20 - 24,53,Finland,220
3,2008,Females,25 - 29,62,Finland,230
4,2008,Females,30 - 34,64,Finland,272


In [23]:
#save to a complete file for futher analysis on death causes in Finland
fin.to_csv("Finland_clean.csv",index=False)

### 3. Clean fin3 dataset

In [24]:
fin3 = pd.read_csv("pop_sex_age.csv")
fin3.head(30)

,Information,Year,Sex,Age,Population 31.12.
0,Population 31 Dec,2008,Males,0 - 4,150804
1,Population 31 Dec,2008,Males,5 - 9,146717
2,Population 31 Dec,2008,Males,10 - 14,157654
3,Population 31 Dec,2008,Males,15 - 19,169961
4,Population 31 Dec,2008,Males,20 - 24,166488
5,Population 31 Dec,2008,Males,25 - 29,174715
6,Population 31 Dec,2008,Males,30 - 34,171532
7,Population 31 Dec,2008,Males,35 - 39,159578
8,Population 31 Dec,2008,Males,40 - 44,186832
9,Population 31 Dec,2008,Males,45 - 49,190839


In [25]:
fin3= fin3.drop(['Information'], axis=1)

In [26]:
fin3 = fin3.rename(columns={"Population 31.12.": "Population"})

In [27]:
fin_age_dict ={
    '0 - 4'  : '0 - 14',
    '5 - 9'  : '0 - 14',
    '10 - 14': '0 - 14',
    '15 - 19': '15 - 19',
    '20 - 24': '20 - 24',
    '25 - 29': '25 - 29',
    '30 - 34': '30 - 34', 
    '35 - 39': '35 - 39', 
    '40 - 44': '40 - 44', 
    '45 - 49': '45 - 49', 
    '50 - 54': '50 - 54', 
    '55 - 59': '55 - 59',
    '60 - 64': '60 - 64', 
    '65 - 69': '65 - 69', 
    '70 - 74': '70 - 74', 
    '75 - 79': '75 - 79', 
    '80 - 84': '80 - 85',
    '85 -'   : '85 - over'
}
fin3['AgeRange_modified'] = fin3['Age'].map(fin_age_dict)
fin3.head()

,Year,Sex,Age,Population,AgeRange_modified
0,2008,Males,0 - 4,150804,0 - 14
1,2008,Males,5 - 9,146717,0 - 14
2,2008,Males,10 - 14,157654,0 - 14
3,2008,Males,15 - 19,169961,15 - 19
4,2008,Males,20 - 24,166488,20 - 24


In [28]:
fin3 = fin3.drop(['Age'], axis=1)

In [29]:
fin3 = fin3.groupby(["Year", "Sex", "AgeRange_modified"]).sum("Population").reset_index()
fin3.head(40)

,Year,Sex,AgeRange_modified,Population
0,2008,Females,0 - 14,435987
1,2008,Females,15 - 19,163216
2,2008,Females,20 - 24,158952
3,2008,Females,25 - 29,165701
4,2008,Females,30 - 34,162543
5,2008,Females,35 - 39,152352
6,2008,Females,40 - 44,181293
7,2008,Females,45 - 49,187140
8,2008,Females,50 - 54,191006
9,2008,Females,55 - 59,198198


In [30]:
fin3.columns = ["Year_1", "Sex", "Age", "population_by_gender_year_age"]
fin3.head()

,Year_1,Sex,Age,population_by_gender_year_age
0,2008,Females,0 - 14,435987
1,2008,Females,15 - 19,163216
2,2008,Females,20 - 24,158952
3,2008,Females,25 - 29,165701
4,2008,Females,30 - 34,162543


In [31]:
fin3.to_csv("Finland_pop_by_gender_year.csv",index=False)

### 4. Merge fin3 and fin dataset

In [32]:
result = pd.concat([fin, fin3], axis=1)
result.tail()

,Year,Gender,Age Range,Deaths,Country,Total,Year_1,Sex,Age,population_by_gender_year_age
411,2020,Males,65 - 69,2602,Finland,4089,2020,Males,65 - 69,169845
412,2020,Males,70 - 74,4024,Finland,6482,2020,Males,70 - 74,165833
413,2020,Males,75 - 79,3702,Finland,6401,2020,Males,75 - 79,103742
414,2020,Males,80 - 84,4606,Finland,8844,2020,Males,80 - 85,65005
415,2020,Males,85 - over,7697,Finland,21933,2020,Males,85 - over,49799


In [33]:
result = result.drop(["Sex", "Age", "Year_1"], axis=1)

In [34]:
result.columns = ["Year", "Gender", "Age Range", "deaths_by_year_gender_age", "Country", "deaths_by_year_age", "population_by_gender_year_age"]
result.head()

,Year,Gender,Age Range,deaths_by_year_gender_age,Country,deaths_by_year_age,population_by_gender_year_age
0,2008,Females,0 - 14,106,Finland,265,435987
1,2008,Females,15 - 19,38,Finland,138,163216
2,2008,Females,20 - 24,53,Finland,220,158952
3,2008,Females,25 - 29,62,Finland,230,165701
4,2008,Females,30 - 34,64,Finland,272,162543


In [35]:
result = result[["Year", "Country", "Age Range", "Gender", "deaths_by_year_gender_age", "deaths_by_year_age", "population_by_gender_year_age"]]
result.head()

,Year,Country,Age Range,Gender,deaths_by_year_gender_age,deaths_by_year_age,population_by_gender_year_age
0,2008,Finland,0 - 14,Females,106,265,435987
1,2008,Finland,15 - 19,Females,38,138,163216
2,2008,Finland,20 - 24,Females,53,220,158952
3,2008,Finland,25 - 29,Females,62,230,165701
4,2008,Finland,30 - 34,Females,64,272,162543


In [36]:
result.to_csv("Fin_clean_for_ASMR.csv",index=False)

### 5. Clean Ger dataset

In [215]:
ger = pd.read_csv("Germany_raw.csv", on_bad_lines='skip',sep=None)
ger.info()
ger.head()

C:\Users\Big City Girl\AppData\Local\Temp\ipykernel_18980\874801166.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  ger = pd.read_csv("Germany_raw.csv", on_bad_lines='skip',sep=None)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38376 entries, 0 to 38375
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   statistics_code         38376 non-null  int64 
 1   statistics_label        38376 non-null  object
 2   time_code               38376 non-null  object
 3   time_label              38376 non-null  object
 4   time                    38376 non-null  int64 
 5   1_variable_code         38376 non-null  object
 6   1_variable_label        38376 non-null  object
 7   1_variable_code.1       38376 non-null  object
 8   1_variable_code.2       38376 non-null  object
 9   2_variable_code         38376 non-null  object
 10  2_variable_label        38376 non-null  object
 11  2_variable_code.1       38376 non-null  object
 12  2_variable_code.2       38376 non-null  object
 13  3_variable_code         38376 non-null  object
 14  3_variable_label        38376 non-null  object
 15  3_

,statistics_code,statistics_label,time_code,time_label,time,1_variable_code,1_variable_label,1_variable_code.1,1_variable_code.2,2_variable_code,...,2_variable_code.2,3_variable_code,3_variable_label,3_variable_code.1,3_variable_code.2,4_variable_code,4_variable_label,4_variable_code.1,4_variable_code.2,BEV002__Deaths__number
0,23211,Causes of death statistics,JAHR,Year,2008,DINSG,Germany,DG,Germany,TODUR1,...,Certain infectious and parasitic diseases,GES,Sex,GESM,Male,ALT027,Age groups,ALT000,under 1 year,23
1,23211,Causes of death statistics,JAHR,Year,2008,DINSG,Germany,DG,Germany,TODUR1,...,Certain infectious and parasitic diseases,GES,Sex,GESM,Male,ALT027,Age groups,ALT001B15,1 to under 15 years,30
2,23211,Causes of death statistics,JAHR,Year,2008,DINSG,Germany,DG,Germany,TODUR1,...,Certain infectious and parasitic diseases,GES,Sex,GESM,Male,ALT027,Age groups,ALT015B20,15 to under 20 years,7
3,23211,Causes of death statistics,JAHR,Year,2008,DINSG,Germany,DG,Germany,TODUR1,...,Certain infectious and parasitic diseases,GES,Sex,GESM,Male,ALT027,Age groups,ALT020B25,20 to under 25 years,13
4,23211,Causes of death statistics,JAHR,Year,2008,DINSG,Germany,DG,Germany,TODUR1,...,Certain infectious and parasitic diseases,GES,Sex,GESM,Male,ALT027,Age groups,ALT025B30,25 to under 30 years,35


In [216]:
ger = ger.drop(["statistics_code","statistics_label","time_code","time_label", "1_variable_code",\
                "1_variable_code.1", "1_variable_code.2", "2_variable_code", "2_variable_label",\
                "3_variable_code", "3_variable_label", "3_variable_code.1", "4_variable_code", \
                "4_variable_label"], axis=1)

In [217]:
ger.head()

,time,1_variable_label,2_variable_code.1,2_variable_code.2,3_variable_code.2,4_variable_code.1,4_variable_code.2,BEV002__Deaths__number
0,2008,Germany,TODESURS02,Certain infectious and parasitic diseases,Male,ALT000,under 1 year,23
1,2008,Germany,TODESURS02,Certain infectious and parasitic diseases,Male,ALT001B15,1 to under 15 years,30
2,2008,Germany,TODESURS02,Certain infectious and parasitic diseases,Male,ALT015B20,15 to under 20 years,7
3,2008,Germany,TODESURS02,Certain infectious and parasitic diseases,Male,ALT020B25,20 to under 25 years,13
4,2008,Germany,TODESURS02,Certain infectious and parasitic diseases,Male,ALT025B30,25 to under 30 years,35


In [218]:
# match age range code to age range name.
ger_new = ger.groupby(["4_variable_code.1", "4_variable_code.2"]).size()
print (ger_new)

4_variable_code.1  4_variable_code.2   
ALT000             under 1 year            2132
ALT001B15          1 to under 15 years     2132
ALT015B20          15 to under 20 years    2132
ALT020B25          20 to under 25 years    2132
ALT025B30          25 to under 30 years    2132
ALT030B35          30 to under 35 years    2132
ALT035B40          35 to under 40 years    2132
ALT040B45          40 to under 45 years    2132
ALT045B50          45 to under 50 years    2132
ALT050B55          50 to under 55 years    2132
ALT055B60          55 to under 60 years    2132
ALT060B65          60 to under 65 years    2132
ALT065B70          65 to under 70 years    2132
ALT070B75          70 to under 75 years    2132
ALT075B80          75 to under 80 years    2132
ALT080B85          80 to under 85 years    2132
ALT085UM           85 years and over       2132
ALTNN              age unknown             2132
dtype: int64


In [219]:
# drop age range specification, leaving only code in dataframe
ger =ger.drop("4_variable_code.2", axis=1)

# drop unknown age rows
ger = ger[ger["4_variable_code.1"] != "ALTNN"] 

In [220]:
# rename and rearrange columns
ger.columns = ["Year", "Country", "Death code", "Reason", "Gender","Age range", "Deaths"]
ger = ger[["Country", "Year", "Gender", "Age range", "Death code", "Reason", "Deaths"]]
ger.head()

,Country,Year,Gender,Age range,Death code,Reason,Deaths
0,Germany,2008,Male,ALT000,TODESURS02,Certain infectious and parasitic diseases,23
1,Germany,2008,Male,ALT001B15,TODESURS02,Certain infectious and parasitic diseases,30
2,Germany,2008,Male,ALT015B20,TODESURS02,Certain infectious and parasitic diseases,7
3,Germany,2008,Male,ALT020B25,TODESURS02,Certain infectious and parasitic diseases,13
4,Germany,2008,Male,ALT025B30,TODESURS02,Certain infectious and parasitic diseases,35


In [221]:
# change deaths variable types
ger["Deaths"]=pd.to_numeric(ger.Deaths, errors='coerce').convert_dtypes() 
ger.head(50)

,Country,Year,Gender,Age range,Death code,Reason,Deaths
0,Germany,2008,Male,ALT000,TODESURS02,Certain infectious and parasitic diseases,23
1,Germany,2008,Male,ALT001B15,TODESURS02,Certain infectious and parasitic diseases,30
2,Germany,2008,Male,ALT015B20,TODESURS02,Certain infectious and parasitic diseases,7
3,Germany,2008,Male,ALT020B25,TODESURS02,Certain infectious and parasitic diseases,13
4,Germany,2008,Male,ALT025B30,TODESURS02,Certain infectious and parasitic diseases,35
5,Germany,2008,Male,ALT030B35,TODESURS02,Certain infectious and parasitic diseases,44
6,Germany,2008,Male,ALT035B40,TODESURS02,Certain infectious and parasitic diseases,103
7,Germany,2008,Male,ALT040B45,TODESURS02,Certain infectious and parasitic diseases,156
8,Germany,2008,Male,ALT045B50,TODESURS02,Certain infectious and parasitic diseases,230
9,Germany,2008,Male,ALT050B55,TODESURS02,Certain infectious and parasitic diseases,286


In [222]:
ger.isnull().sum()

Country          0
Year             0
Gender           0
Age range        0
Death code       0
Reason           0
Deaths        6337
dtype: int64

In [223]:
# modify age range to match Finnish dataset
ger_age_dict ={
    'ALT000'     : '0 - 14',
    'ALT001B15'  : '0 - 14',
    'ALT015B20'  : '15 - 19',
    'ALT020B25'  : '20 - 24',
    'ALT025B30'  : '25 - 29',
    'ALT030B35'  : '30 - 34', 
    'ALT035B40'  : '35 - 39', 
    'ALT040B45'  : '40 - 44', 
    'ALT045B50'  : '45 - 49', 
    'ALT050B55'  : '50 - 54', 
    'ALT055B60'  : '55 - 59',
    'ALT060B65'  : '60 - 64', 
    'ALT065B70'  : '65 - 69', 
    'ALT070B75'  : '70 - 74', 
    'ALT075B80'  : '75 - 79', 
    'ALT080B85'  : '80 - 84',
    'ALT085UM'   : '85 - over'
}
ger['AgeRange_modified'] = ger['Age range'].map(ger_age_dict)


In [224]:
# fill NA in death column = 0 
ger["Deaths"]=ger['Deaths'].fillna(0)
ger.isnull().sum()

Country              0
Year                 0
Gender               0
Age range            0
Death code           0
Reason               0
Deaths               0
AgeRange_modified    0
dtype: int64

In [225]:
ger = ger.groupby(["Year", "Gender", "AgeRange_modified"]).sum("Deaths").reset_index()

In [226]:
# rename column name
ger["Country"] ="Germany"
ger = ger[["Country", "Year", "Gender", "AgeRange_modified", "Deaths"]]
ger.head()


,Country,Year,Gender,AgeRange_modified,Deaths
0,Germany,2008,Female,0 - 14,4055
1,Germany,2008,Female,15 - 19,1270
2,Germany,2008,Female,20 - 24,1668
3,Germany,2008,Female,25 - 29,2058
4,Germany,2008,Female,30 - 34,2450


In [227]:
ger['Total'] = ger.groupby(["Year", "AgeRange_modified"], sort=False)['Deaths'].transform('sum')
ger.head(20)

,Country,Year,Gender,AgeRange_modified,Deaths,Total
0,Germany,2008,Female,0 - 14,4055,9346
1,Germany,2008,Female,15 - 19,1270,4441
2,Germany,2008,Female,20 - 24,1668,6385
3,Germany,2008,Female,25 - 29,2058,7037
4,Germany,2008,Female,30 - 34,2450,8210
5,Germany,2008,Female,35 - 39,5406,15277
6,Germany,2008,Female,40 - 44,11607,32378
7,Germany,2008,Female,45 - 49,21131,58300
8,Germany,2008,Female,50 - 54,29846,85140
9,Germany,2008,Female,55 - 59,41573,119384


In [228]:
ger.columns = ["Country", "Year", "Gender", "Age Range", "Deaths", "Total"]
ger.head(50)

,Country,Year,Gender,Age Range,Deaths,Total
0,Germany,2008,Female,0 - 14,4055,9346
1,Germany,2008,Female,15 - 19,1270,4441
2,Germany,2008,Female,20 - 24,1668,6385
3,Germany,2008,Female,25 - 29,2058,7037
4,Germany,2008,Female,30 - 34,2450,8210
5,Germany,2008,Female,35 - 39,5406,15277
6,Germany,2008,Female,40 - 44,11607,32378
7,Germany,2008,Female,45 - 49,21131,58300
8,Germany,2008,Female,50 - 54,29846,85140
9,Germany,2008,Female,55 - 59,41573,119384


In [229]:
ger = ger[["Year", "Gender", "Age Range", "Deaths", "Country", "Total"]]
ger.head()

,Year,Gender,Age Range,Deaths,Country,Total
0,2008,Female,0 - 14,4055,Germany,9346
1,2008,Female,15 - 19,1270,Germany,4441
2,2008,Female,20 - 24,1668,Germany,6385
3,2008,Female,25 - 29,2058,Germany,7037
4,2008,Female,30 - 34,2450,Germany,8210


In [ ]:
#save to a complete file for futher analysis on death causes in Finland
ger.to_csv("Germany_clean.csv",index=False)


### 6. Clean Ger_2_raw dataset

In [192]:
ger2 = pd.read_csv("Ger_2_raw.csv")
ger2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1136 entries, 0 to 1135
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Age          1131 non-null   object 
 1   Male         1131 non-null   float64
 2   Female       1131 non-null   float64
 3   Total        1131 non-null   float64
 4   Year         1131 non-null   float64
 5   Country      1131 non-null   object 
 6   Unnamed: 6   0 non-null      float64
 7   Unnamed: 7   0 non-null      float64
 8   Unnamed: 8   0 non-null      float64
 9   Unnamed: 9   0 non-null      float64
 10  Unnamed: 10  0 non-null      float64
dtypes: float64(9), object(2)
memory usage: 97.8+ KB


In [193]:
ger2= ger2.drop(["Unnamed: 6", "Unnamed: 7", "Unnamed: 8", "Unnamed: 9", "Unnamed: 10"], axis=1)

In [194]:
ger2.head()

,Age,Male,Female,Total,Year,Country
0,under 1 year,351137.0,333241.0,684378.0,2008.0,Ger
1,1 year,350400.0,331728.0,682128.0,2008.0,Ger
2,2 years,350611.0,332338.0,682949.0,2008.0,Ger
3,3 years,358384.0,340306.0,698690.0,2008.0,Ger
4,4 years,363590.0,345252.0,708842.0,2008.0,Ger


In [195]:
ger2["Age"].unique()

array(['under 1 year', '1 year', '2 years', '3 years', '4 years',
       '5 years', '6 years', '7 years', '8 years', '9 years', '10 years',
       '11 years', '12 years', '13 years', '14 years', '15 years',
       '16 years', '17 years', '18 years', '19 years', '20 years',
       '21 years', '22 years', '23 years', '24 years', '25 years',
       '26 years', '27 years', '28 years', '29 years', '30 years',
       '31 years', '32 years', '33 years', '34 years', '35 years',
       '36 years', '37 years', '38 years', '39 years', '40 years',
       '41 years', '42 years', '43 years', '44 years', '45 years',
       '46 years', '47 years', '48 years', '49 years', '50 years',
       '51 years', '52 years', '53 years', '54 years', '55 years',
       '56 years', '57 years', '58 years', '59 years', '60 years',
       '61 years', '62 years', '63 years', '64 years', '65 years',
       '66 years', '67 years', '68 years', '69 years', '70 years',
       '71 years', '72 years', '73 years', '74 years', '

In [196]:
# modify age range 
ger2_age_dict ={
    'under 1 year'  : '0 - 14',
    '1 year'   : '0 - 14',
    '2 years'  : '0 - 14',
    '3 years'  : '0 - 14',
    '4 years'  : '0 - 14',
    '5 years'  : '0 - 14',
    '6 years'  : '0 - 14',
    '7 years'  : '0 - 14',
    '8 years'  : '0 - 14',
    '9 years'  : '0 - 14',
    '10 years'  : '0 - 14',
    '11 years'  : '0 - 14',
    '12 years'  : '0 - 14',
    '13 years'  : '0 - 14',
    '14 years'  : '0 - 14',
    '15 years'  : '15 - 19',
    '16 years'   : '15 - 19',
    '17 years'  : '15 - 19',
    '18 years'  : '15 - 19',
    '19 years'  : '15 - 19',
    '20 years'  : '20 - 24',
    '21 years'  : '20 - 24',
    '22 years'  : '20 - 24',
    '23 years'  : '20 - 24',
    '24 years'  : '20 - 24',
    '25 years'  : '25 - 29',
    '26 years'  : '25 - 29',
    '27 years'  : '25 - 29',
    '28 years'  : '25 - 29',
    '29 years'  : '25 - 29',
    '30 years': '30 - 34',
    '31 years': '30 - 34',
    '32 years': '30 - 34',
    '33 years': '30 - 34',
    '34 years': '30 - 34', 
    '35 years': '35 - 39', 
    '36 years': '35 - 39', 
    '37 years': '35 - 39', 
    '38 years': '35 - 39', 
    '39 years': '35 - 39', 
    '40 years': '40 - 44', 
    '41 years': '40 - 44',
    '42 years': '40 - 44',
    '43 years': '40 - 44',
    '44 years': '40 - 44',
    '45 years': '45 - 49', 
    '46 years': '45 - 49', 
    '47 years': '45 - 49', 
    '48 years': '45 - 49', 
    '49 years': '45 - 49', 
    '50 years': '50 - 54', 
    '51 years': '50 - 54', 
    '52 years': '50 - 54', 
    '53 years': '50 - 54', 
    '54 years': '50 - 54', 
    '55 years': '55 - 59',
    '56 years': '55 - 59',
    '57 years': '55 - 59',
    '58 years': '55 - 59',
    '59 years': '55 - 59',
    '60 years': '60 - 64', 
    '61 years': '60 - 64', 
    '62 years': '60 - 64', 
    '63 years': '60 - 64', 
    '64 years': '60 - 64', 
    '65 years': '65 - 69', 
    '66 years': '65 - 69', 
    '67 years': '65 - 69', 
    '68 years': '65 - 69', 
    '69 years': '65 - 69', 
    '70 years': '70 - 74', 
    '71 years': '70 - 74', 
    '72 years': '70 - 74', 
    '73 years': '70 - 74', 
    '74 years': '70 - 74', 
    '75 years': '75 - 79', 
    '76 years': '75 - 79', 
    '77 years': '75 - 79', 
    '78 years': '75 - 79', 
    '79 years': '75 - 79', 
    '80 years': '80 - 84',
    '81 years': '80 - 84',
    '82 years': '80 - 84',
    '83 years': '80 - 84',
    '84 years': '80 - 84',
    '85 years and over': '85 - over',
    'Total': "Total"
}
ger2['AgeRange_modified'] = ger2['Age'].map(ger2_age_dict)
ger2.tail(50)

,Age,Male,Female,Total,Year,Country,AgeRange_modified
1086,42 years,499939.0,493171.0,993110.0,2020.0,Ger,40 - 44
1087,43 years,491952.0,486657.0,978609.0,2020.0,Ger,40 - 44
1088,44 years,480675.0,475754.0,956429.0,2020.0,Ger,40 - 44
1089,45 years,476741.0,472515.0,949255.0,2020.0,Ger,45 - 49
1090,46 years,479804.0,478008.0,957811.0,2020.0,Ger,45 - 49
1091,47 years,501260.0,498488.0,999748.0,2020.0,Ger,45 - 49
1092,48 years,547189.0,541714.0,1088903.0,2020.0,Ger,45 - 49
1093,49 years,582617.0,576276.0,1158893.0,2020.0,Ger,45 - 49
1094,50 years,614755.0,606141.0,1220895.0,2020.0,Ger,50 - 54
1095,51 years,651105.0,640184.0,1291289.0,2020.0,Ger,50 - 54


In [197]:
ger2 = ger2.loc[ger2.Age != "Total"].reset_index()

In [198]:
ger2=ger2.drop(["index"], axis=1)

In [199]:
ger2.tail(20)

,Age,Male,Female,Total,Year,Country,AgeRange_modified
1103,71 years,398562.0,447793.0,846355.0,2020.0,Ger,70 - 74
1104,72 years,363057.0,411701.0,774758.0,2020.0,Ger,70 - 74
1105,73 years,321985.0,370954.0,692939.0,2020.0,Ger,70 - 74
1106,74 years,274110.0,322987.0,597097.0,2020.0,Ger,70 - 74
1107,75 years,292841.0,350799.0,643640.0,2020.0,Ger,75 - 79
1108,76 years,330639.0,399047.0,729686.0,2020.0,Ger,75 - 79
1109,77 years,319752.0,390920.0,710672.0,2020.0,Ger,75 - 79
1110,78 years,336632.0,421743.0,758374.0,2020.0,Ger,75 - 79
1111,79 years,362167.0,467686.0,829853.0,2020.0,Ger,75 - 79
1112,80 years,350651.0,465752.0,816403.0,2020.0,Ger,80 - 84


In [200]:
ger2=ger2[:1118]
ger2.tail(20)

,Age,Male,Female,Total,Year,Country,AgeRange_modified
1098,66 years,475038.0,519282.0,994319.0,2020.0,Ger,65 - 69
1099,67 years,460781.0,509294.0,970075.0,2020.0,Ger,65 - 69
1100,68 years,450637.0,501566.0,952203.0,2020.0,Ger,65 - 69
1101,69 years,440871.0,493423.0,934294.0,2020.0,Ger,65 - 69
1102,70 years,428684.0,481252.0,909935.0,2020.0,Ger,70 - 74
1103,71 years,398562.0,447793.0,846355.0,2020.0,Ger,70 - 74
1104,72 years,363057.0,411701.0,774758.0,2020.0,Ger,70 - 74
1105,73 years,321985.0,370954.0,692939.0,2020.0,Ger,70 - 74
1106,74 years,274110.0,322987.0,597097.0,2020.0,Ger,70 - 74
1107,75 years,292841.0,350799.0,643640.0,2020.0,Ger,75 - 79


In [201]:
ger2_modified=ger2.melt(id_vars=["Age", "Year", "Country", "AgeRange_modified"], 
        var_name="gen", 
        value_name="pop_by_gender_age_year")
ger2_modified.tail(50)

,Age,Year,Country,AgeRange_modified,gen,pop_by_gender_age_year
3304,36 years,2020.0,Ger,35 - 39,Total,1049492.0
3305,37 years,2020.0,Ger,35 - 39,Total,1059922.0
3306,38 years,2020.0,Ger,35 - 39,Total,1065149.0
3307,39 years,2020.0,Ger,35 - 39,Total,1062390.0
3308,40 years,2020.0,Ger,40 - 44,Total,1038503.0
3309,41 years,2020.0,Ger,40 - 44,Total,1005308.0
3310,42 years,2020.0,Ger,40 - 44,Total,993110.0
3311,43 years,2020.0,Ger,40 - 44,Total,978609.0
3312,44 years,2020.0,Ger,40 - 44,Total,956429.0
3313,45 years,2020.0,Ger,45 - 49,Total,949255.0


In [202]:
ger2_modified= ger2_modified.loc[ger2_modified.gen != "Total"].reset_index()

In [203]:
ger2_modified.tail(20)

,index,Age,Year,Country,AgeRange_modified,gen,pop_by_gender_age_year
2216,2216,66 years,2020.0,Ger,65 - 69,Female,519282.0
2217,2217,67 years,2020.0,Ger,65 - 69,Female,509294.0
2218,2218,68 years,2020.0,Ger,65 - 69,Female,501566.0
2219,2219,69 years,2020.0,Ger,65 - 69,Female,493423.0
2220,2220,70 years,2020.0,Ger,70 - 74,Female,481252.0
2221,2221,71 years,2020.0,Ger,70 - 74,Female,447793.0
2222,2222,72 years,2020.0,Ger,70 - 74,Female,411701.0
2223,2223,73 years,2020.0,Ger,70 - 74,Female,370954.0
2224,2224,74 years,2020.0,Ger,70 - 74,Female,322987.0
2225,2225,75 years,2020.0,Ger,75 - 79,Female,350799.0


In [204]:
ger2_modified=ger2_modified.drop(["index", "Age"], axis=1)

In [205]:
ger2_modified.head(20)

,Year,Country,AgeRange_modified,gen,pop_by_gender_age_year
0,2008.0,Ger,0 - 14,Male,351137.0
1,2008.0,Ger,0 - 14,Male,350400.0
2,2008.0,Ger,0 - 14,Male,350611.0
3,2008.0,Ger,0 - 14,Male,358384.0
4,2008.0,Ger,0 - 14,Male,363590.0
5,2008.0,Ger,0 - 14,Male,367366.0
6,2008.0,Ger,0 - 14,Male,374781.0
7,2008.0,Ger,0 - 14,Male,386870.0
8,2008.0,Ger,0 - 14,Male,396585.0
9,2008.0,Ger,0 - 14,Male,402202.0


In [206]:
ger2_modified = ger2_modified.groupby(["Year", "gen", "AgeRange_modified"]).sum("pop_by_gender_age_year").reset_index()
ger2_modified.tail(20)

,Year,gen,AgeRange_modified,pop_by_gender_age_year
396,2020.0,Female,70 - 74,2034687.0
397,2020.0,Female,75 - 79,2030195.0
398,2020.0,Female,80 - 84,1961089.0
399,2020.0,Female,85 - over,1624163.0
400,2020.0,Male,0 - 14,5874037.0
401,2020.0,Male,15 - 19,2015167.0
402,2020.0,Male,20 - 24,2405361.0
403,2020.0,Male,25 - 29,2594396.0
404,2020.0,Male,30 - 34,2853603.0
405,2020.0,Male,35 - 39,2685998.0


In [209]:
ger2_modified=ger2_modified.astype({'Year': 'int64', "pop_by_gender_age_year": "int64"})

In [210]:
ger2_modified.head(20)

,Year,gen,AgeRange_modified,pop_by_gender_age_year
0,2008,Female,0 - 14,5458973
1,2008,Female,15 - 19,2224055
2,2008,Female,20 - 24,2395355
3,2008,Female,25 - 29,2459976
4,2008,Female,30 - 34,2320419
5,2008,Female,35 - 39,2856585
6,2008,Female,40 - 44,3469624
7,2008,Female,45 - 49,3341439
8,2008,Female,50 - 54,2907154
9,2008,Female,55 - 59,2683665


In [213]:
ger2_modified.columns = ["Year_2", "Sex", "Age", "pop_by_gender_year_age"]
ger2_modified.head()

,Year_2,Sex,Age,pop_by_gender_year_age
0,2008,Female,0 - 14,5458973
1,2008,Female,15 - 19,2224055
2,2008,Female,20 - 24,2395355
3,2008,Female,25 - 29,2459976
4,2008,Female,30 - 34,2320419


In [214]:
ger2_modified.to_csv("Germany_pop_by_gender_year.csv",index=False)

### 7. Merge ger and ger2_modified dataframes

In [233]:
result_1 = pd.concat([ger, ger2_modified], axis=1)
result_1.head(50)

,Year,Gender,Age Range,Deaths,Country,Total,Year_2,Sex,Age,pop_by_gender_year_age
0,2008,Female,0 - 14,4055,Germany,9346,2008,Female,0 - 14,5458973
1,2008,Female,15 - 19,1270,Germany,4441,2008,Female,15 - 19,2224055
2,2008,Female,20 - 24,1668,Germany,6385,2008,Female,20 - 24,2395355
3,2008,Female,25 - 29,2058,Germany,7037,2008,Female,25 - 29,2459976
4,2008,Female,30 - 34,2450,Germany,8210,2008,Female,30 - 34,2320419
5,2008,Female,35 - 39,5406,Germany,15277,2008,Female,35 - 39,2856585
6,2008,Female,40 - 44,11607,Germany,32378,2008,Female,40 - 44,3469624
7,2008,Female,45 - 49,21131,Germany,58300,2008,Female,45 - 49,3341439
8,2008,Female,50 - 54,29846,Germany,85140,2008,Female,50 - 54,2907154
9,2008,Female,55 - 59,41573,Germany,119384,2008,Female,55 - 59,2683665


In [235]:
result_1 = result_1.drop(["Sex", "Age", "Year_2"], axis=1)

In [236]:
result_1.columns = ["Year", "Gender", "Age Range", "deaths_by_year_gender_age", "Country", "deaths_by_year_age", "population_by_gender_year_age"]
result_1.head()

,Year,Gender,Age Range,deaths_by_year_gender_age,Country,deaths_by_year_age,population_by_gender_year_age
0,2008,Female,0 - 14,4055,Germany,9346,5458973
1,2008,Female,15 - 19,1270,Germany,4441,2224055
2,2008,Female,20 - 24,1668,Germany,6385,2395355
3,2008,Female,25 - 29,2058,Germany,7037,2459976
4,2008,Female,30 - 34,2450,Germany,8210,2320419


In [237]:
result_1 = result_1[["Year", "Country", "Age Range", "Gender", "deaths_by_year_gender_age", "deaths_by_year_age", "population_by_gender_year_age"]]
result_1.head()

,Year,Country,Age Range,Gender,deaths_by_year_gender_age,deaths_by_year_age,population_by_gender_year_age
0,2008,Germany,0 - 14,Female,4055,9346,5458973
1,2008,Germany,15 - 19,Female,1270,4441,2224055
2,2008,Germany,20 - 24,Female,1668,6385,2395355
3,2008,Germany,25 - 29,Female,2058,7037,2459976
4,2008,Germany,30 - 34,Female,2450,8210,2320419


In [238]:
result_1.to_csv("Ger_clean_for_ASMR.csv",index=False)